In [3]:
import nglview as ngl
import ipywidgets as widgets
from IPython.display import HTML, display
import nglview as nv
import numpy as np
from IPython.display import display
from ipywidgets import HTML, VBox, Output


descriptions = {
    'Cas9 protein': 'The Cas9 protein used in this model is a visual representation of the SpyCas9 protein used in Casgevy. Casgevy specifically uses a mutation of this protein, with one such mutation being the K855A mutation, in which the amino acid Lysine (K) is replaced with the amino acid Alanine (A) at position 855. This helps with enabling the enzyme to be more effective in cutting the target DNA.',
    'Guide RNA': 'The Guide RNA is the sequence responsible for working with the Cas9 protein to attack the target DNA strand. The Guide RNA sequence is complementary to the target DNA sequence so that it would bind to the target DNA sequence and guide the Cas9 to the desired area.',
    'PAM Sequence': 'The PAM Sequence acts like a landmark for the Cas9 protein. It is a small DNA sequence responsible for leading the Cas9 protein to the correct destination. Located next to the target DNA sequence, it is required for the Cas9 protein to work.',
    'Target DNA Strand': 'The Target DNA strand is the strand that needs editing. Located next to the PAM sequence, the Cas9 protein edits this sequence by cutting out the undesirable qualities within it.',
    'Active Sites': 'These are the locations on the Cas9 proteins responsible for activities related to enzymes such as RNA processing and DNA cleavage.',
    'Cut Sites': 'When editing the target DNA, the Cas9 protein cuts 3 base pairs upstream of the PAM sequence, leading to these double-stranded breaks in the DNA, known as the Cut sites.',
    'Non-Target DNA strand': 'The Non-Target DNA strand is the strand that will not be targeted by the Cas9.'  
}   

representation_selector = widgets.Dropdown(
    options=['Cas9 protein', 'Guide RNA', 'PAM Sequence', 'Target DNA Strand', 'Active Sites', 'Cut Sites', 'Non-Target DNA strand' ],
    description='Representation:'
)       

view = nv.NGLWidget()

description_html = """
<div class="simulation-description">
    <h1>CRISPR Cas-9 Simulation of Casgevy</h1>
    
    <h2>Simulation Details</h2>
    <p>This is an imperfect model of a CRISPR Cas-9 model regarding the use of Casgevy to fight Sickle Cell Disease.</p> 


    <h2>Purpouse of this model</h2>
    <p>The purpose of this model is to do some digging into the latest technology that can potenitally cure sickle cell disease. It's meant to show how the Guide RNA and the Cas9 protein interact with the target dna to edit genes. </p>
     <h3>Interaction Instructions:</h3>
    <ul>
        <li><strong>Mouse Zoom</strong>: Use mouse wheel or right-click + drag up/down</li>
        <li><strong>Touch Zoom</strong>: Pinch to zoom in/out</li>
        <li><strong>Rotation</strong>: Left-click + drag (mouse) or one-finger drag (touch)</li>
        <li><strong>Pan</strong>: Middle-click + drag (mouse) or two-finger drag (touch)</li>
    </ul>
</div>
"""
style = """
<style>
.description {
    font-family: Arial, sans-serif;
    padding: 10px;
    background-color: #f8f8f8;
    border-radius: 5px;
    margin-bottom: 10px;
}
</style>
"""
description_widget = widgets.HTML(
    value=description_html,
    layout=widgets.Layout(margin='10px 0px')
    )
representation_dropdown = widgets.Dropdown(
    options=['cartoon', 'ball+stick', 'licorice', 'surface'],
    value='cartoon',
    description='Style:',
    layout=widgets.Layout(width='200px')
)
color_scheme = widgets.Dropdown(
    options=['element', 'residue', 'chain', 'sstruc'],
    value='chain',
    description='Color:',
    layout=widgets.Layout(width='200px')
)
def update_view(change):
    view.clear_representations()
    view.add_representation(
        representation_dropdown.value,
        selection='all',
        color_scheme=color_scheme.value
    )
representation_dropdown.observe(update_view, names='value')
color_scheme.observe(update_view, names='value')
update_view(None)
controls = widgets.HBox([representation_dropdown, color_scheme])

zoom_slider = widgets.FloatSlider(
    value=40,
    min=5,
    max=100,
    step=1,
    description='Zoom:',
    layout=widgets.Layout(width='300px')
)
reset_button = widgets.Button(
    description='Reset View',
    icon='undo',
    layout=widgets.Layout(width='120px')
)
def update_representation(change):
    view.clear_representations()
    view.add_representation(
        representation_dropdown.value,
        selection='all',
        color_scheme=color_scheme.value
    )
def update_zoom(change):
    view._set_camera_zoom(zoom_slider.value)
def reset_view(b):
    view._reset_view()
    zoom_slider.value = 40

representation_dropdown.observe(update_representation, names='value')
color_scheme.observe(update_representation, names='value')
zoom_slider.observe(update_zoom, names='value')
reset_button.on_click(reset_view)
update_representation(None)
style_controls = widgets.HBox([representation_dropdown, color_scheme])
zoom_controls = widgets.HBox([zoom_slider, reset_button])
style_container = widgets.Box(
    [style_controls], 
    layout=widgets.Layout(
        margin='10px 0px',
        padding='10px',
        border='1px solid #ddd',
        border_radius='5px',
        background_color='#eee'
    )
)
zoom_container = widgets.Box(
    [zoom_controls], 
    layout=widgets.Layout(
        margin='10px 0px',
        padding='10px',
        border='1px solid #ddd',
        border_radius='5px',
        background_color='#eee'
    )
) 
mobile_instructions = widgets.HTML(
    value="""<div class="mobile-instructions">
        <p><strong>Mobile users:</strong> Use pinch gestures to zoom, one finger to rotate, and two fingers to pan.</p>
    </div>""",
    layout=widgets.Layout(margin='5px 0')
)
style_widget = widgets.HTML(value=style)
app = widgets.VBox([
    style_widget,
    description_widget,
    representation_dropdown,
    style_container,
    zoom_container,
    mobile_instructions,
    view
])

description_label = widgets.Label(value=descriptions[representation_selector.value])
cas9_complex = view.add_component('4oo8.pdb')



def on_representation_change(change):
    if change['name'] == 'value':
        description_label.value = descriptions[change['new']]
        
        view.clear_representations()
       
        if change['new'] == 'Cas9 protein':
            cas9_complex.add_representation('cartoon', selection='protein', color='skyblue', 
                               opacity=0.8, name='Cas9 protein')
            cas9_complex.add_representation('surface', selection='protein', color='skyblue', 
                               opacity=0.3, name='Cas9 surface')

        elif change['new'] == 'Guide RNA':
             cas9_complex.add_representation('cartoon', selection='rna', color='green', 
                               name='Guide RNA')
             cas9_complex.add_representation('base', selection='rna', color='green', 
                               name='RNA bases')

        elif change['new'] == 'Target DNA Strand':
             cas9_complex.add_representation('cartoon', selection='dna and chain C', color='red', 
                               name='Target DNA strand')
            
        elif change['new'] == 'Non-Target DNA strand':
             cas9_complex.add_representation('cartoon', selection='dna and chain D', color='orange', 
                               name='Non-target DNA strand')

        elif change['new'] == 'Cut Sites':
             cas9_complex.add_representation('ball+stick', selection='dna and (nucleic and backbone) and (chain C and resno 12-13)', 
                               color='magenta', name='Cut site')

        elif change['new'] == 'PAM Sequence':
             cas9_complex.add_representation('licorice', selection='dna and chain D and resno 20-22', 
                               color='yellow', name='PAM sequence')

        elif change['new'] == 'Active Sites':
             cas9_complex.add_representation('ball+stick', selection='protein and (resno 10 or resno 840 or resno 854)', 
                               color='purple', name='Catalytic residues')
        
        

representation_selector.observe(on_representation_change)

view.camera = 'perspective'
view.center()
view.spin = False

view._remote_call('setLabel', target='Widget', args=['Cas9-sgRNA complex editing DNA'])

display(view)

style_widget = widgets.HTML(value=style)



app = widgets.VBox([
    description_widget,
    description_label,
    view,
    HTML(value="<b>Click on drop down menu to see different parts of the model:</b>"),
    representation_selector, 
])

display(app)

NGLWidget()

In [ ]:
!pip freeze-requirements.txt